# Advanced Modeling of Groundwater Flow (GW3099)
![alt text](..\img\header.jpg)

# Exercise 1: MODFLOW 6
The purpose of this exercise is to use read and plot the MODFLOW 6 results using Flopy.

In [ ]:
from __future__ import print_function
%matplotlib inline
import sys
import os
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy

import config

print(sys.version)
print('python executable: {}'.format(sys.executable))
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

In [ ]:
model_ws = './ex01-completed'
s = flopy.mf6.MFSimulation().load(sim_ws=model_ws)
modelname = 'ex01' #s.model_names[0]

In [ ]:
print('Model name is {}'.format(modelname))

In [ ]:
gwf = s.get_model(modelname)

In [ ]:
nlay = gwf.dis.nlay.get_data()
nrow = gwf.dis.nrow.get_data()
ncol = gwf.dis.ncol.get_data()
print(nlay, nrow, ncol)

In [ ]:
gwf.dis.top = 450

In [ ]:
gwf.dis.top.array

In [ ]:
gwf.riv

In [ ]:
# run the model, if it hasn't been run yet
if not os.path.isfile(os.path.join(model_ws, modelname + '.dis.grb')):
    s.run_simulation()

In [ ]:
# Load binary grid file
fname = os.path.join(model_ws, modelname + '.dis.grb')
grd = flopy.utils.MfGrdFile(fname, verbose=False)
sr = grd.get_spatialreference()

# load the heads and budget files
fname = os.path.join(model_ws, modelname + '.hds')
hdobj = flopy.utils.HeadFile(fname)
head = hdobj.get_data()

fname = os.path.join(model_ws, modelname + '.cbc')
cbb = flopy.utils.CellBudgetFile(fname, precision='double')
spdis = cbb.get_data(text='DATA-SPDIS')[0]
qx = spdis['qx'].reshape((nlay, nrow, ncol))
qy = spdis['qy'].reshape((nlay, nrow, ncol))
xg = sr.xcentergrid
yg = sr.ycentergrid

In [ ]:
# plot the results for each layer
fig, axes = plt.subplots(nrows=nlay, ncols=1, figsize=(10, 15))
vmin = head.min()
vmax = head.max()
levels = np.linspace(vmin, vmax, 20)

for ilay in range(nlay):
    ax = axes[ilay]
    #ax = fig.add_subplot(1, nlay, ilay + 1, aspect='equal')
    ax.set_title('Layer {}'.format(ilay + 1))
    ax.set_aspect('equal')
    modelmap = flopy.plot.ModelMap(sr=sr, layer=ilay, ax=ax)
    quadmesh = modelmap.plot_array(head, masked_values=[999.], alpha=0.5, 
                                   vmin=vmin, vmax=vmax, cmap='jet')
    linecollection = modelmap.plot_grid()
    contour_set = modelmap.contour_array(head, levels=levels, colors='white')
    #quiver = modelmap.plot_discharge(frf, fff, head=head)
    ax.quiver(xg, yg, qx[ilay], qy[ilay])
cb = fig.colorbar(quadmesh, ax=axes, shrink=0.5)